In [54]:
    import pandas as pd
    from datetime import time

In [69]:
    #Load Data Sets
    pd_links = pd.read_csv('dataSets/training/links (table 3).csv')
    pd_routes = pd.read_csv('dataSets/training/routes (table 4).csv')
    pd_trajectories = pd.read_csv('dataSets/training/trajectories(table 5)_training.csv')
    pd_volume = pd.read_csv('dataSets/training/volume(table 6)_training.csv')

In [70]:
    #先处理trajectories数据
    pd_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [73]:
    #字符转为时间格式
    pd_trajectories['starting_time'] = pd.to_datetime(pd_trajectories['starting_time'], format = '%Y-%m-%d %H:%M:%S')
    
    #获取日期
    pd_trajectories['date'] = pd_trajectories['starting_time'].dt.date
    
    #获取时间
    pd_trajectories['time'] = pd_trajectories['starting_time'].dt.time
    
    #获取小时
    def get_hour(t):
        return t.hour
    
    pd_trajectories['hour'] = pd_trajectories['time'].apply(get_hour)
    
    #获取时间段
    def get_timewindow(t):
        delta = 20
        if t.minute < delta:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif time.minute < delta*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window
    
    pd_trajectories['time_window'] = pd_trajectories['time'].apply(get_timewindow)
    
    #获取星期
    pd_trajectories['weekday'] = pd_trajectories['starting_time'].dt.dayofweek + 1  #pandas的周一从0开始，周日为6
    
    #获取是否为法定假日(国庆和中秋）
    holiday1 = [pd.Timestamp('2016-10-1'), pd.Timestamp('2016-10-8')]
    holiday2 = [pd.Timestamp('2016-9-15'), pd.Timestamp('2016-9-18')]
    
    pd_trajectories['holiday'] = pd_trajectories['starting_time'].between(holiday1[0],holiday1[1])\
                                    | pd_trajectories['starting_time'].between(holiday2[0],holiday2[1])
    
    #删除'time'字段
    del pd_trajectories['time']
    
    pd_trajectories.head()

TypeError: drop() takes at least 2 arguments (1 given)

In [66]:
    #获取links
    def get_links(seq):
        l_seq = seq.split(';')
        links = []
        for s in l_seq:
            link = s.split('#')[0]
            links.append(link)
        s_links = ';'.join(links)
        return s_links
    
    pd_trajectories['links'] = pd_trajectories['travel_seq'].apply(get_links)

In [67]:
    pd_trajectories.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time,date,time,hour,time_window,weekday,holiday,links
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85,2016-07-19,00:14:24,0,"[00:00:00,00:20:00)",2,False,105;100;111;103;122
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79,2016-07-19,00:35:56,0,"[00:40:00,01:00:00)",2,False,105;100;111;103;122
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76,2016-07-19,00:37:15,0,"[00:40:00,01:00:00)",2,False,105;100;111;103;116;101;121;106;113
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05,2016-07-19,00:37:59,0,"[00:40:00,01:00:00)",2,False,110;123;107;108;120;117
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98,2016-07-19,00:56:21,0,"[00:40:00,01:00:00)",2,False,105;100;111;103;116;101;121;106;113


In [68]:
    pd_trajectories.to_csv('Processed_data_by_weikai/processed_train_trajectories.csv')